# Analysen mit Pivottabellen
[Pivottabellen](https://de.wikipedia.org/wiki/Pivot-Tabelle) (auch *Kreuztabllen*) finden sich häufig in Analysen, etwa basierend auf Daten eines *Data Warehouse* und sind ein fester Bestandteil des *Business Reporting*. Zunächst beschäftigen wir uns daher kurz mit einigen zugrundeliegenden Konzepten.

## Konzepte
> Die folgenden Abbildungen sind größtenteils der [Dissertation](http://webdoc.sub.gwdg.de/ebook/dissts/Oldenburg/Herden2001.pdf) von [Prof. Dr. Olaf Herden](http://www.ba-horb.de/~he) entnommen.

Die Daten in einem Data Warehouse werden häufig als [*Datenwürfel*](https://de.wikipedia.org/wiki/OLAP-W%C3%BCrfel) (engl. *data cube*) modelliert, d.h., sie haben eine *multi-dimensionale* Struktur mit *Dimensionshierarchien* und enthalten als Werte die zugehörigen *Fakten*.

<!---
![Datenwürfel](img/bild08-datacube.png)
--->
<img src="img/bild08-datacube.png" width="600">

In relationalen Datenbanken wird dies oft mit sog. [*Sternschemata*](https://de.wikipedia.org/wiki/Sternschema) abgebildet. In einem solchen befindet sich in der Mitte die sog. *Faktentabelle* und um diese herum die *Dimensionstabellen*. So könnte ein zu obigem Datenwürfel passendes Sternschema als ER-Modell wie folgt aussehen: 


![Sternschema](img/bild16-star.png)
<!---
<img src="img/bild16-star.png" width="600">
--->

Hierbei sind folgende *Dimensionshierarchien* zu erkennen:
1. eine *Ortshierarchie* in `Filiale`<br/>
Von unten nach oben lautet die Hierarchie `Bezeichnung` &rarr; `Stadt` &rarr; `Region` &rarr; `Land`. Im Würfel
wäre eine solche Bezeichnung *OL West*, in der Stadt *Oldenburg*, in der Region *Niedersachsen* im Land *Deutschland*.
2. eine *Zeithierarchie* in `Zeit`<br/>
Genauer gesagt gibt es hier zwei Hierarchien, zum einen `Tag` &rarr; `Woche` &rarr; `Jahr` und zum anderen
`Tag` &rarr; `Monat` &rarr; `Quartal` &rarr; `Jahr` (da eine Kalenderwoche in zwei verschiedenen Monaten oder auch Quartalen liegen kann). Das Datum *21.12.1999* aus dem Würfel hätte genau diesen Tag (oder einfach die *21*), die Woche wäre *51/1999* (oder nur *51*), der Monat *12/1999* (oder nur *12*), das Quartal *4/1999* (oder nur *4*) und das Jahr *1999*.
3. spezifische Hierarchien in `Produkt`<br/>
Auch hier gibt es zwei Hierarchien, nämlich `Bezeichnung` &rarr; `Marke` &rarr; `Hersteller` und `Bezeichnung` &rarr; `Produktgruppe`. Letztere ließe sich ggf. noch um `Produkthauptgruppe` als nächsthöhere Stufe erweitern.

Derartige Hierarchien bilden dann sog. *Verdichtungspfade*, entlang derer man die Fakten immer weiter *aggregieren* kann. Im obigen Beispiel würden die Basisfakten, die aussagen, wie viele eines Produktes an einem bestimmten Tag in einer Filiale verkauft wurden, dann vermutlich mit der *Aggregatfunktion Summe* verdichtet, beispielsweise zu einer Übersicht, wie viele Produkte einer Produktgruppe in einem bestimmten Jahr in einer Region verkauft wurden.

Genao solche aggregierten Daten werden im Allgemeinen werden in Pivottabellen dargestellt, wobei man die Daten durch Filterbedingungen weiter einschränken kann. Bleibt man bei der Würfelmetapher, so sind die Pivottabellen im mathematischen Sinne *zweidimensionale Projektionen* (von Teilen) des Würfels.

Neben den vorgenannten Konzepten, die die *multi-dimensionale Datenhaltung* betreffen, gibt es weitere Konzepte, die den *Umgang* mit diesen Daten unter Berücksichtigung der Struktur betreffen, nämlich die *multi-dimensionalen Operationen*.

## Multi-dimensionale Operationen

Die wichtigsten multi-dimensionalen Operationen sind
1. Drill-down und Roll-up<br/>
Hier geht es um *Verfeinerung* (beim Drill-down) bzw. *Verdichtung* (beim Roll-up) innerhalb einer Dimensionshierarchie.
2. Split und Merge<br/>
Hier erfolgt die *Verfeinerung* durch Hinzunahme von Attributen aus einer *anderen Dimension (bzw. Hierarchie)* (beim Split) und die *Verdichtung* (Merge) erfolgt durch Weglassen eines solchen Attributs.
3. Slicing und Dicing<br/>
Hierbei werden *Attributwerte gefiltert*, d.h., man trifft eine (Vor-)Auswahl wie mit einer `WHERE`-Klausel in SQL. Beim Slicing werden dabei nur Werte eines einzigen Attributs eingeschränkt, beim Dicing die mehrerer Attribute.
4. Pivoting<br/>
Hierbei wird der *Blickwinkel* auf den Datenwürfel verändert, d.h. man stellt andere Attribute als zuvor einander gegenüber.

Diese Operationen wollen wir zunächst kurz konzeptionell darstellen.

### Drill-down und Roll-up

Im folgenden Bild erfolgt die Aggregation (Roll-up) in der Zeithierarchie von Monaten auf Quartale bzw. die Verfeinerung (Drill-down) umgekehrt von Quartalen auf Monate.

<!---
![Drill](img/bild10-rollup.png)
--->
<img src="img/bild10-rollup.png" width="450">


### Split und Merge

Optisch sind diese Operationen nicht von Drill-down und Roll-up zu unterscheiden. Nehmen wir an, wir hätten die Regionen X, Y und Z und wir wollten die Quartalszahlen des oberen Würfels nach diesen Regionen aufgeteilt (gesplittet) darstellen. Dann könnte der untere Würfel gerade so aussehen wie der untere im obigen Bild, nur dass die Zahl ganz unten links in der Ecke eben den verkauften Einheiten in der Region X im Quartal I entspräche und die ganz unten rechts den verkauften Einheiten in der Region Z im Quartal IV.

** Zu beachten:** Im Rahmen des Business Reporting wird meist nur von Drill-down und Roll-up gesprochen, da praktisch kein Unterschied zu Split und Merge zu sehen ist. Konzeptionell gibt es allerdings sehr wohl einen wichtigen Unterschied, denn bei Drill-down und Roll-up gibt es zwischen den involvierten Attributen eine *hierarchische Beziehung*, während die Attribute bei Split und Merge *voneinander unabhängig* sind (was durchaus auch praktische Auswirkungen hat, wie wir unten bei der Implementierung sehen werden).

### Slicing und Dicing

<!---
![SliceDice](img/bild12-slicedice.png)
--->
<img src="img/bild12-slicedice.png" width="800">


Die beiden linken Bilder zeigen Slicing, die beiden rechten Dicing. Dabei wurde im ersten Bild ganz links ein spezieller Wert für den Ort ausgewählt (also wohl eine bestimmte Filiale), was zu einer sehr dünnen Scheibe (engl. *slice*) des Würfels führt. Im zweiten Bild wird dann ein *Bereich* von Produktwerten ausgewählt, was zu einer dickeren Scheibe führt. In beiden Fällen werden aber jeweils nur innerhalb eines Attributs Werte gefiltert.

Im dritten Bild wurde ein Zeitpunkt sowie Bereiche bezüglich Produkt und Ort festgelegt und im letzten Bild ganz rechts Bereiche für alle drei Dimensionen, was zu einem kleinen Würfel (engl. *dice*) innerhalb des gesamten Datenwürfels führt.

Somit ist Slicing natürlich lediglich ein *Spezialfall* des Dicing. Allerdings wird meistens nur der Begriff Slicing für beides verwendet, zumal die Unterscheidung auch rein theoretischer Natur ist.

** Zu beachten:** In der Praxis kommen all diese Operationen (Drill-down, Roll-Up, Split, Merge und Slicing und Dicing) natürlich nicht nur in Reinform, sondern meist in Kombination vor. Dennoch können Darstellungen in Pivottabellen immer als Kombination der vorgenannten multi-dimensionalen Operationen angesehen werden, solange man multi-dimensional strukturierte Daten betrachtet. Das ist aber zumindest implizit immer der Fall, da sonst die Präsentation in Pivottabellen sinnlos wäre.

### Pivoting
Abschließend sei noch das Pivoting erwähnt, dem die von uns hier betrachteten Analysewerkzeuge, die Pivottabellen, ihren Namen verdanken. Wie bereits erwähnt geht es dabei darum, den Würfel aus verschiedenen Perspektiven zu betrachten. Dies kann dadurch geschehen, dass man einen *Angelpunkt* (frz. *pivot*) festhält (im Bild *orange* markiert). Dann dreht man den Würfel (im Bild anhand des *blauen Punktes* verdeutlicht) und sieht dann eine andere zweidimensionale Projektion des Würfels, wenn man immer weiter von vorne draufschaut.

<!---
![Pivoting](img/bild13-pivot.png)
--->
<img src="img/bild13-pivot.png" width="600">


Nun wollen wir uns noch ansehen, wie diese Konzepte mit Pandas umgesetzt werden können.

## Implementierung in Pandas
In Pandas gibt es für Data Frames die Methoden [`pivot_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.pivot_table.html) und [`crosstab`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.crosstab.html). Eine allgemeinere Beschreibung findet man unter dem Stichwort [*Reshaping*](http://pandas.pydata.org/pandas-docs/stable/reshaping.html). Zunächst importieren wir wieder die benötigten Bibliotheken.

In [1]:
import numpy as np
import pandas as pd
# Ausgabe langer Tabellen auf 20 Zeilen beschränken (und später zurück setzen)
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20

Unsere Daten befinden sich in einer SQLite-Datenbank (damit die Intereesierten probieren können, die Ergebnisse auch per SQL zu erzeugen, falls gewünscht).

In [2]:
from pandas.io import sql
import sqlite3

conn = sqlite3.connect('data/pivtab.db')
query = "SELECT * FROM absatz;"

absatz = sql.read_sql(query, con=conn, parse_dates=True)
absatz.head()

,Datum,Tag,Monat,Jahr,Standort,Produkthauptgruppe,Produktgruppe,Menge
0,2002-01-01,1,1,2002,BY,Schreibwaren,Papier,10
1,2002-01-01,1,1,2002,BW,Schreibwaren,Stifte,1
2,2002-01-01,1,1,2002,RP,Lebensmittel,Gebäck,12
3,2002-01-01,1,1,2002,BY,Lebensmittel,Gebäck,1
4,2002-01-01,1,1,2002,BY,Schreibwaren,Stifte,8


In [ ]:
# Alternativ könnte man auch direkt die csv-Datei einlesen
#absatz = pd.read_csv('data/pivtab.csv', sep=';', parse_dates=True)
#absatz

Diese Absatztabelle könnte also ein Auszug aus der im obigen Sternschema dargestellten Datenbank sein, wobei neben der Produktgruppe nun auch noch eine Produkthauptgruppe ergänzt wurde. Wir schauen zunächst, mit welcher Art von Daten wir es zu tun habn.

In [3]:
absatz.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6487 entries, 0 to 6486
Data columns (total 8 columns):
Datum                 6487 non-null object
Tag                   6487 non-null int64
Monat                 6487 non-null int64
Jahr                  6487 non-null int64
Standort              6487 non-null object
Produkthauptgruppe    6487 non-null object
Produktgruppe         6487 non-null object
Menge                 6487 non-null int64
dtypes: int64(4), object(4)
memory usage: 405.5+ KB


In [4]:
absatz.describe()

,Tag,Monat,Jahr,Menge
count,6487.000000,6487.000000,6487.000000,6487.000000
mean,15.747033,6.562510,2002.993680,6.987668
std,8.836769,3.442143,0.815244,3.753568
min,1.000000,1.000000,2002.000000,1.000000
25%,8.000000,4.000000,2002.000000,4.000000
50%,16.000000,7.000000,2003.000000,7.000000
75%,23.000000,10.000000,2004.000000,10.000000
max,31.000000,12.000000,2004.000000,13.000000


Nun wollen wir mit den Analysen beginnen.

### Eine erste Pivottabelle

Zunächst wollen wir einfach die Absatzmengen der Produkthauptgruppen nach Standorten darstellen, d.h., die gewünschte Pivottabelle sollte wie folgt aussehen (bis auf das Filterelement "Jahr"; dazu kommen wir später):

<!---
![Erste Pivottabelle](img/bild17-pivtab.png)
--->
<img src="img/bild17-pivtab.png" width="600">

Wir beginnen mit der Ermittlung der Grunddaten (d.h. der Werte in den weißen Zellen).

In [5]:
absatz.pivot_table('Menge', index=['Produkthauptgruppe'], columns=['Standort'], aggfunc='sum')

Standort,BW,BY,HS,RP
Produkthauptgruppe,,,,
Lebensmittel,3633,3978,3694,3714
Schreibwaren,3750,3653,3584,3910
Zeitschriften,4210,3704,3886,3613


Das zugehörige SQL-Statement wäre:
```SQL
SELECT Produkthauptgruppe, Standort, SUM(Menge)
FROM absatz
GROUP BY Produkthauptgruppe, Standort;
```
Allerdings würde dies zu einer *gestapelten* Ausgabe führen, wobei die Zeilennamen (also der *Index*) die erste und die Spaltennamen die zweite Spalte bilden. D.h., die Ausgabe wäre (fast) so, wie die des folgenden Python-Codes (es würden im SQL-Ergebnis lediglich die Namen der Produkthaupgruppen immer wiederholt):

In [6]:
pd.crosstab(absatz.Produkthauptgruppe, absatz.Standort, values=absatz.Menge, aggfunc=np.sum).stack()

Produkthauptgruppe  Standort
Lebensmittel        BW          3633
                    BY          3978
                    HS          3694
                    RP          3714
Schreibwaren        BW          3750
                    BY          3653
                    HS          3584
                    RP          3910
Zeitschriften       BW          4210
                    BY          3704
                    HS          3886
                    RP          3613
dtype: int64

Interessanter Weise führt auch die Verwendung von `unstack()` dazu, dass eine gestapelte Ausgabe erfolgt. Allerdings steht dabei die Spalte mit den Spaltennamen vorne, und die Spalte mit den Zeilennamen kommt als zweite:

In [7]:
absatz.pivot_table('Menge', index=['Produkthauptgruppe'], columns=['Standort'], aggfunc='sum').unstack()

Standort  Produkthauptgruppe
BW        Lebensmittel          3633
          Schreibwaren          3750
          Zeitschriften         4210
BY        Lebensmittel          3978
          Schreibwaren          3653
          Zeitschriften         3704
HS        Lebensmittel          3694
          Schreibwaren          3584
          Zeitschriften         3886
RP        Lebensmittel          3714
          Schreibwaren          3910
          Zeitschriften         3613
dtype: int64

Daran ist zu erkennen, dass die Darstellung als Pivottabelle von Pandas als eine spezielle Art der Darstellung einer Stapelung aufgefasst wird.

#### Summenwerte

Nun fehlen zu der gewünschten Pivottabelle noch die Summenzeile bzw. die Summenspalte sowie die Gesamtsumme. In SQL wäre das aufwändig (es sei denn, das DBMS unterstützt die SQL/OLAP Erweiterungen, z.B. Oracle). In SQLite müsste man etwa folgende Abfrage schreiben:

```SQL
-- weiße Zellen (Grunddaten)
SELECT Produkthauptgruppe, Standort, SUM(Menge)
FROM absatz
GROUP BY Produkthauptgruppe, Standort
UNION
-- Summierung über die Produkthauptgruppen
SELECT "All", Standort, SUM(Menge)
FROM absatz
GROUP BY Standort
UNION
-- Summierung über die Standorte
SELECT Produkthauptgruppe, "All", SUM(Menge)
FROM absatz
GROUP BY Produkthauptgruppe
UNION
-- Gesamtsumme
SELECT "All", "All", SUM(Menge)
FROM absatz
```

In Pandas ist dies wesentlich leichter, denn man benötigt lediglich die Spezifikation `margins = True`.

In [8]:
absatz.pivot_table('Menge', index=['Produkthauptgruppe'], columns=['Standort'], aggfunc='sum', margins=True)

Standort,BW,BY,HS,RP,All
Produkthauptgruppe,,,,,
Lebensmittel,3633.0,3978.0,3694.0,3714.0,15019.0
Schreibwaren,3750.0,3653.0,3584.0,3910.0,14897.0
Zeitschriften,4210.0,3704.0,3886.0,3613.0,15413.0
All,11593.0,11335.0,11164.0,11237.0,45329.0


Allerdings ist zu sehen, dass aus den ganzzahligen Werten jetzt *Dezimalzahlen* geworden sind!

Nun schauen wir uns an, wie die multi-dimensionalen Operation mit Pandas ausgeführt werden können.

### Drill-down und Roll-up
Wir möchten eine (erweiterte) Pivottabelle wie die im folgenden Bild erzeugen.

<!---
![DrillPivottabelle](img/bild18-extpivtab_rollup.png)
--->
<img src="img/bild18-extpivtab_rollup.png" width="700">


Hier sieht man, dass die jeweiligen Zwischensummen für die einzelnen Produkthauptgruppen die Verdichtung (das Roll-up) der Werte für die Produktgruppen sind bzw. umgekehrt die Werte für die Produkthauptgruppen auf die Produktgruppen heruntergebrochen werden (Drill-down). Schauen wir, was mit Pandas passiert:

In [9]:
absatz.pivot_table('Menge', index=['Produkthauptgruppe','Produktgruppe'], columns=['Standort'], 
                     aggfunc='sum', margins=True)

Standort                               BW       BY       HS       RP      All
Produkthauptgruppe Produktgruppe                                             
Lebensmittel       Gebäck          1872.0   1810.0   1769.0   2010.0   7461.0
                   Süsses          1761.0   2168.0   1925.0   1704.0   7558.0
Schreibwaren       Papier          1959.0   1829.0   1889.0   2071.0   7748.0
                   Stifte          1791.0   1824.0   1695.0   1839.0   7149.0
Zeitschriften      Magazin         2156.0   2089.0   1993.0   1767.0   8005.0
                   Tageszeitung    2054.0   1615.0   1893.0   1846.0   7408.0
All                               11593.0  11335.0  11164.0  11237.0  45329.0

**Beobachtung:** Leider werden die *Zwischensummen nicht* berechnet und es gibt auch keinen einfachen Weg, dies zu tun! Das ist eine gewisse Schwäche der API.

Ferner wäre es hier wegen der hierarchischen Beziehung der Attribute sinnlos, die Indexreihenfolge unzukehren (also die Reihenfolge der Zeilenfelder), wie man am folgenden Beispiel sieht:

In [10]:
absatz.pivot_table('Menge', index=['Produktgruppe','Produkthauptgruppe'], columns=['Standort'], 
                     aggfunc='sum', margins=True)

,Standort,BW,BY,HS,RP,All
Produktgruppe,Produkthauptgruppe,,,,,
Gebäck,Lebensmittel,1872.0,1810.0,1769.0,2010.0,7461.0
Magazin,Zeitschriften,2156.0,2089.0,1993.0,1767.0,8005.0
Papier,Schreibwaren,1959.0,1829.0,1889.0,2071.0,7748.0
Stifte,Schreibwaren,1791.0,1824.0,1695.0,1839.0,7149.0
Süsses,Lebensmittel,1761.0,2168.0,1925.0,1704.0,7558.0
Tageszeitung,Zeitschriften,2054.0,1615.0,1893.0,1846.0,7408.0
All,,11593.0,11335.0,11164.0,11237.0,45329.0


### Split und Merge

Analog zum vorherigen Fall soll nun die folgende (erweiterte) Pivottabelle generiert werden.
<!---
![SplitPivottabelle](img/bild19-extpivtab_split.png)
--->
<img src="img/bild19-extpivtab_split.png" width="700">

Auch hier sieht man, dass die Ergebnisse für die Produkthauptgruppen die Ergebnisse der einzelnen Jahre zusammenführen (Merge) bzw. diese auf die einzelnen Jahre aufgeteilt werden (Split). Der zugehörige Python-Code gleicht dem vorherigen bis auf den zweiten Index:

In [11]:
absatz.pivot_table('Menge', index=['Produkthauptgruppe','Jahr'], columns=['Standort'], 
                     aggfunc='sum', margins=True)

Standort                      BW       BY       HS       RP      All
Produkthauptgruppe Jahr                                             
Lebensmittel       2002   1242.0   1493.0   1310.0   1374.0   5419.0
                   2003   1273.0   1262.0   1272.0   1325.0   5132.0
                   2004   1118.0   1223.0   1112.0   1015.0   4468.0
Schreibwaren       2002   1194.0   1066.0   1240.0   1155.0   4655.0
                   2003   1256.0   1300.0   1181.0   1483.0   5220.0
                   2004   1300.0   1287.0   1163.0   1272.0   5022.0
Zeitschriften      2002   1370.0   1034.0   1440.0   1243.0   5087.0
                   2003   1441.0   1343.0   1317.0   1079.0   5180.0
                   2004   1399.0   1327.0   1129.0   1291.0   5146.0
All                      11593.0  11335.0  11164.0  11237.0  45329.0

Wieder sieht man, dass keine Zwischensummen gebildet werden. Hier wäre es nun allerdings durchaus möglich, die Reihenfolge der Attribute im Index umzukehren, da beide voneinander unabhängig sind:

In [12]:
absatz.pivot_table('Menge', index=['Jahr','Produkthauptgruppe'], columns=['Standort'], 
                     aggfunc='sum', margins=True)

Standort                      BW       BY       HS       RP      All
Jahr Produkthauptgruppe                                             
2002 Lebensmittel         1242.0   1493.0   1310.0   1374.0   5419.0
     Schreibwaren         1194.0   1066.0   1240.0   1155.0   4655.0
     Zeitschriften        1370.0   1034.0   1440.0   1243.0   5087.0
2003 Lebensmittel         1273.0   1262.0   1272.0   1325.0   5132.0
     Schreibwaren         1256.0   1300.0   1181.0   1483.0   5220.0
     Zeitschriften        1441.0   1343.0   1317.0   1079.0   5180.0
2004 Lebensmittel         1118.0   1223.0   1112.0   1015.0   4468.0
     Schreibwaren         1300.0   1287.0   1163.0   1272.0   5022.0
     Zeitschriften        1399.0   1327.0   1129.0   1291.0   5146.0
All                      11593.0  11335.0  11164.0  11237.0  45329.0

Der konzeptionelle Unterschied zwischen Drill-down und Roll-up sowie Split und Merge hat also zwar gewisse praktische Relevanz. Dennoch hat sich im Sprachgebrauch nur die Verwendung von Drill-down und Roll-up für beide Fälle durchgesetzt.

### Slicing und Dicing

In einer ersten Gegenüberstellung möchten wir die Absatzmengen der einzelnen Jahre an den Standorten für die Produkthauptgruppe Schreibwaren sehen.

<!---
![SliceDicePivottabelle](img/bild20-slicedice.png)
--->
<img src="img/bild20-slicedice.png" width="700">

In einer zweiten dann die Absatzmengen für Lebensmittel und Zeitschriften an den Standorten in Bayern und Bade-Württemberg im Jahr 2003. 

Wir beginnen mit der ersten, die ein reines Slicing ist.

In [13]:
slicer = absatz.Produkthauptgruppe == 'Schreibwaren'
absatz[slicer].pivot_table('Menge', index=['Jahr'], columns=['Standort'], 
                     aggfunc='sum', margins=True)

Standort,BW,BY,HS,RP,All
Jahr,,,,,
2002,1194.0,1066.0,1240.0,1155.0,4655.0
2003,1256.0,1300.0,1181.0,1483.0,5220.0
2004,1300.0,1287.0,1163.0,1272.0,5022.0
All,3750.0,3653.0,3584.0,3910.0,14897.0


Die zweite ist zwar genau genommen ein *Dicing*, aber dieser Begriff wird meist nicht verwendet. Daher werden wir die logische Variable für die Filterung weiter `slicer` nennen.
<!---
siehe hier: https://stackoverflow.com/questions/12065885/filter-dataframe-rows-if-value-in-column-is-in-a-set-list-of-values
--->

In [14]:
slicer = (absatz.Produkthauptgruppe.isin(['Lebensmittel','Zeitschriften']))
slicer = slicer & (absatz.Standort.isin(['BW','BY'])) & (absatz.Jahr == 2003)
absatz[slicer].pivot_table('Menge', index=['Produkthauptgruppe'], columns=['Standort'], 
                     aggfunc='sum', margins=True)

Standort,BW,BY,All
Produkthauptgruppe,,,
Lebensmittel,1273.0,1262.0,2535.0
Zeitschriften,1441.0,1343.0,2784.0
All,2714.0,2605.0,5319.0


#### Slicing mit fester Struktur
Hat man eine bestimmte Struktur einer Pivottabelle und möchte hier nur ggf. weitere Filterungen vornehmen (also Slicing betreiben), dann gibt es auch noch eine weitere Möglichkeit dafür. Zunächst speichert man diese Struktur in einem eigenen Data Frame ab.

In [15]:
pt = absatz.pivot_table('Menge', index=['Jahr','Produkthauptgruppe'], columns=['Standort'], 
                     aggfunc='sum', margins=True)
pt

Standort                      BW       BY       HS       RP      All
Jahr Produkthauptgruppe                                             
2002 Lebensmittel         1242.0   1493.0   1310.0   1374.0   5419.0
     Schreibwaren         1194.0   1066.0   1240.0   1155.0   4655.0
     Zeitschriften        1370.0   1034.0   1440.0   1243.0   5087.0
2003 Lebensmittel         1273.0   1262.0   1272.0   1325.0   5132.0
     Schreibwaren         1256.0   1300.0   1181.0   1483.0   5220.0
     Zeitschriften        1441.0   1343.0   1317.0   1079.0   5180.0
2004 Lebensmittel         1118.0   1223.0   1112.0   1015.0   4468.0
     Schreibwaren         1300.0   1287.0   1163.0   1272.0   5022.0
     Zeitschriften        1399.0   1327.0   1129.0   1291.0   5146.0
All                      11593.0  11335.0  11164.0  11237.0  45329.0

Nun können wir wie folgt filtern:

In [16]:
pt.query('Produkthauptgruppe == ["Lebensmittel","Zeitschriften"]')

Standort                     BW      BY      HS      RP     All
Jahr Produkthauptgruppe                                        
2002 Lebensmittel        1242.0  1493.0  1310.0  1374.0  5419.0
     Zeitschriften       1370.0  1034.0  1440.0  1243.0  5087.0
2003 Lebensmittel        1273.0  1262.0  1272.0  1325.0  5132.0
     Zeitschriften       1441.0  1343.0  1317.0  1079.0  5180.0
2004 Lebensmittel        1118.0  1223.0  1112.0  1015.0  4468.0
     Zeitschriften       1399.0  1327.0  1129.0  1291.0  5146.0

In [17]:
pt.query('Produkthauptgruppe == ["Lebensmittel","Zeitschriften"] & Jahr != [2002]')

Standort                     BW      BY      HS      RP     All
Jahr Produkthauptgruppe                                        
2003 Lebensmittel        1273.0  1262.0  1272.0  1325.0  5132.0
     Zeitschriften       1441.0  1343.0  1317.0  1079.0  5180.0
2004 Lebensmittel        1118.0  1223.0  1112.0  1015.0  4468.0
     Zeitschriften       1399.0  1327.0  1129.0  1291.0  5146.0

Leider kann man nur auf Zeilenfelder (Index) filtern und nicht auf Spaltenfelder, wie der folgende Versuch zeigt, der zu einem Fehler führt:

In [18]:
pt.query('Standort == ["BW","BY"]')

IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match

### Pivoting

Das konzeptionelle Drehen des Datenwürfels lässt sich praktisch einfach durch Ersetzen einer Dimension durch eine andere realisieren. Wir erzeugen zunächst eine Pivottabelle, in der die Jahre dem Standort gegenüber gestellt werden.

In [19]:
absatz.pivot_table('Menge', index=['Jahr'], columns=['Standort'], aggfunc='sum', margins=True)

Standort,BW,BY,HS,RP,All
Jahr,,,,,
2002,3806.0,3593.0,3990.0,3772.0,15161.0
2003,3970.0,3905.0,3770.0,3887.0,15532.0
2004,3817.0,3837.0,3404.0,3578.0,14636.0
All,11593.0,11335.0,11164.0,11237.0,45329.0


Nun pivotieren wir diese, indem wir statt der Jahre die Produkthauptgruppen betrachten (womit wir wieder zu unserer ersten Pivottabelle oben kommen):

In [20]:
absatz.pivot_table('Menge', index=['Produkthauptgruppe'], columns=['Standort'], aggfunc='sum', margins=True)

Standort,BW,BY,HS,RP,All
Produkthauptgruppe,,,,,
Lebensmittel,3633.0,3978.0,3694.0,3714.0,15019.0
Schreibwaren,3750.0,3653.0,3584.0,3910.0,14897.0
Zeitschriften,4210.0,3704.0,3886.0,3613.0,15413.0
All,11593.0,11335.0,11164.0,11237.0,45329.0


Damit haben wir gesehen, wie die Konzepte der multi-dimensionalen Operationen mit Hilfe von Pandas umgesetzt werden können (leider *ohne Zwischensummen*). Allerdings gibt es noch eine weitere, sehr mächtige, vielseitige und elegante Möglichkeit, nämlich die Nutzung der Bibliothek `pivottablejs`.

## Drag & Drop Pivottabellen mit pivottablejs

Es gibt eine sehr einfache Möglichkeit, Pivottabellen per Drag & Drop in Jupyter Notebooks zu erstellen. Diese basiert auf der Javascript-Library [pivottable.js](https://github.com/nicolaskruchten/pivottable) von [Nicolas Kruchten](http://nicolas.kruchten.com). Diese hat der Autor auch als [Python-Library](https://github.com/nicolaskruchten/jupyter_pivottablejs) zur Verfügung und auf seinem [Blog vorgestellt](http://nicolas.kruchten.com/content/2015/09/jupyter_pivottablejs/).

Man kann diese einfach mittels
```
pip install pivottablejs
```
installieren.

**Hinweise**
1. Ursprünglich handelt es sich bei `pivottable.js` um eine Java-Script Library mit eigener [Homepage](https://pivottable.js.org/). Dort findet man auch zahlreiche Beispiele für deren Nutzung.
2. Besonders praktisch ist, dass man dort eigene Datensätze hochladen und diese [direkt im Browser](https://pivottable.js.org/examples/local.html) analysieren kann. Dies können Sie auch als Fallback-Option nutzen, falls die Installation aus irgend einem Grund nicht klappen sollte.

Hat die Installation geklappt, so benötigt man lediglich noch zwei Zeilen Code:

In [21]:
from pivottablejs import pivot_ui
pivot_ui(absatz)

Damit lassen sich nun die Pivottabellen, die wir oben mittels `pivot_table` bzw. `crosstab` generiert hatten, ganz einfach per Drag & Drop erzeugen. Als erstes sollten wir dafür statt `Count` aus der Dropdown-Liste `Integer Sum` auswählen. Daraufhin erscheint eine weitere Dropdown-Liste, aus der wir das Attribut `Menge` auswählen, denn das sind ja die *Fakten*, die wir analysieren möchten. Anschließend wird sofort die Gesamtsumme berechnet.

Um nun die bereits oben betrachteten Pivottabellen zu erhalten, müssen wir noch folgende Korrespondenzen beachten:

`pivot_table` | `pivot_ui`
------------- | ----------
index         | Zeilenfeld
columns       | Spaltenfeld
slicer        | Dropdown-Filter

Im Einzelnen bedeutet dies Folgendes:
1. *Erste Pivottabelle*<br/>
Hier wird `Produkthauptgruppe` zum Zeilen- und `Standort` zum Spaltenfeld
2. *Drill-down und Roll-up*<br/>
Hier muss lediglich `Produktgruppe` als zusätzliches Zeilenfeld *unter* `Produkthauptgruppe` gezogen werden. dies ist zwingend notwendig, da die beiden Attribute in einer *hierarchischen Beziehung* stehen.
3. *Split und Merge*<br/>
`Produktgruppe` als Zeilenfeld wird wieder entfernt und stattdessen `Jahr` verwendet. Hierbei ist zu beachten, dass auch die Umkehrung der Reihenfolge hier möglich ist, da die beiden Attribute voneinander *unabhängig* sind.
4. *Slicing und Dicing*<br/>
Hierfür müssen nur jeweils die gewünschten Werte in den Dropdown-Listen der Attribute ausgewählt werden.

Und nun noch eine Besonderheit: man ist nicht auf die Darstellung als (reine) Tabelle beschränkt. Wählt man beispielsweise in der obersten Dropdown-Liste statt `Tabelle` `Heatmap` aus, so werden die dargestellten Werte sofort mit einer Heatmap unterlegt, was natürlich die Analyse visuell sehr gut unterstützt.

Die Möglichkeiten, die man mit `pivot_ui` hat, werden auf der [Wikiseite auf Github](https://github.com/nicolaskruchten/pivottable/wiki/UI-Tutorial) animiert vorgeführt. Damit hat man eine sehr mächtige Möglichkeit für Business Reporting mit Python! Einschränkend ist auch hier zu erwähnen, dass *keine Zwischensummen* berechnet werden.

## Übung im Unterricht
Erstellen Sie die zuvor betrachteten Pivottabellen mit Heatmaps und probieren Sie weitere Möglichkeiten von `pivottablejs` aus!

## Anregung
Schauen Sie sich an, was der Autor der der `pivottable.js`-Library, Nicolas Kruchten, getan hat, um einem Journalisten zu helfen, einen [Artikel über Beschwerden bei der Montrealer Stadtverwaltung](http://montrealgazette.com/news/local-news/311-calls-whats-bugging-montreal) zu schreiben. Dies erläutert er zum einen auf [seinem Blog](http://nicolas.kruchten.com/content/2015/06/montreal-311/) und auch sehr eindrücklich in der zugehörigen [Data Story](http://nicolas.kruchten.com/mtl311/). 

Dies ist ein schönes Beispiel für angewandte Data Science.

In [ ]:
# Rücksetzen der Anzahl der darzustellenden Zeilen (falls gewünscht)
pd.options.display.max_rows = PREVIOUS_MAX_ROWS